In [ ]:
from pathlib import Path
import matplotlib as mpl

import matplotlib.pyplot as plt
from flow_analysis_comps.Fourier.OrientationSpaceManager import orientationSpaceManager
from flow_analysis_comps.util.video_io import read_video_metadata, load_tif_series_to_dask
import colorcet  # noqa: F401
from flow_analysis_comps.video_manipulation.control_class import videoControl

# mpl.rcParams.update(mpl.rcParamsDefault)

# plt.rcParams["text.usetex"] = True
plt.close("all")

%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
data_adr = Path(
    r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine"
)  # Personal PC path
data_adr = Path(
    r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine"
)  # Personal mac path
data_adr = Path(r"U:\test_data\20250122_Plate017")

plate_id = r"20250122_Plate017"
video_id = r"042"

total_path = data_adr / video_id / "Img"
info_path = next(total_path.parent.glob("videoInfo.*"))

print(info_path)
# info_path = total_path / "videoInfo.json"

video_info = read_video_metadata(info_path)
array = load_tif_series_to_dask(total_path)  # Dims are z, y, x
video_control_obj = videoControl(total_path, info_path, resolution=1)

print(video_info)

In [ ]:
video_control_obj.plot_edge_extraction()

In [ ]:
images = video_control_obj.save_edge_videos(info_path.parent)

In [ ]:
import tifffile
import numpy as np

for edge in video_control_obj.edges:
    fig, ax = plt.subplots()
    ax.imshow(edge.kymograph)
    ax.set_title(edge.edge_info)
    tifffile.imwrite(
        total_path.parent / f"{edge.edge_info}.tiff",
        edge.edge_video,
        imagej=True,
        metadata={"axes": "TYX"},
    )


In [ ]:
OSFilter = orientationSpaceManager(
    .3,
    freq_width=.3,
    K=16,
    x_spacing=video_control_obj.space_pixel_size,
    y_spacing=video_control_obj.time_pixel_size,
)
order = 16
for edge in video_control_obj.edges:
    img = edge.kymograph
    invert_im = video_control_obj.video_info.mode == "brightfield"
    OSFilter.demo_image(
        img,
        video_control_obj.space_pixel_size,
        video_control_obj.time_pixel_size,
        order=order,
        thresh_method="triangle",
        invert=invert_im,
        histo_thresh=.1,
        speed_extent=10
    )


## Testing the AOS filter on segmenting the hyphae themselves

In [ ]:
import numpy as np


def rebin(arr: np.ndarray, new_shape):
    shape = (
        new_shape[0],
        arr.shape[0] // new_shape[0],
        new_shape[1],
        arr.shape[1] // new_shape[1],
    )
    return arr.reshape(shape).mean(-1).mean(1)


OSFilterSegment = orientationSpaceManager(1, K=12)
print(OSFilterSegment.filter.params.freq_width)
order = 12
img = video_control_obj.mean_img
img = img.max() - img
nx, ny = img.shape
print(nx, ny)
img = rebin(img, (nx // 15, ny // 16))


OSFilter.demo_image(img, order=order, thresh_method="otsu")